In [8]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import csv

In [9]:
response = requests.get("https://www.seed-db.com/accelerators")
txt = response.text
acc_soup = BeautifulSoup(txt, 'html.parser')
data = []

In [10]:
def get_acc(soup):
    table=soup.find(id="accellist")
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols=row.find_all('td')
        counter=0

        for cols in row:
            try:
                cols.find('span')
            except NavigableString:
                pass
        col3=None
        col1=row.find('strong')
        links=row.find_all('a')
        links=[ele['href'] for ele in links]
        #print(links)
        if links:
            fu,col2=links[0].split("=")
            if len(links)==2:
                col3=links[1]
        
        data.append([col1.text.strip(),col2,col3])
    return data
get_acc(acc_soup)

[['10-xcelerator',
  '1001',
  'http://fisher.osu.edu/centers/entrepreneurship/10x/'],
 ['Betaspring', '1002', 'http://betaspring.com'],
 ['Excelerate Labs', '1003', None],
 ['FounderFuel', '1004', 'http://founderfuel.com/'],
 ['HAXLR8R', '1005', 'http://haxlr8r.com'],
 ['i/o ventures', '1006', None],
 ['iVentures10', '1007', None],
 ['Portland Seed Fund', '1008', 'http://portlandseedfund.com/'],
 ['Summer @ Highland', '1009', 'http://www.hcp.com/summer/'],
 ['Triangle Startup Factory', '1010', 'http://trianglestartupfactory.com/'],
 ['Y Combinator', '1011', 'http://ycombinator.com'],
 ['500startups', '2001', 'http://500.co'],
 ['AngelPad', '2002', 'http://angelpad.org/'],
 ['Blueprint Health', '2003', 'http://blueprinthealth.org'],
 ['The Brandery', '2004', 'http://brandery.org'],
 ['Entrepreneurs Roundtable Accelerator', '2005', 'http://eranyc.com/'],
 ['Flashpoint', '2006', 'http://flashpoint.gatech.edu/'],
 ['NDRC (LaunchPad / VentureLab)', '2007', 'http://www.ndrc.ie'],
 ['Shotput

In [11]:
print(type(data))
data = pd.DataFrame(data)
data

<class 'list'>


,0,1,2
0,10-xcelerator,1001,http://fisher.osu.edu/centers/entrepreneurship...
1,Betaspring,1002,http://betaspring.com
2,Excelerate Labs,1003,None
3,FounderFuel,1004,http://founderfuel.com/
4,HAXLR8R,1005,http://haxlr8r.com
5,i/o ventures,1006,None
6,iVentures10,1007,None
7,Portland Seed Fund,1008,http://portlandseedfund.com/
8,Summer @ Highland,1009,http://www.hcp.com/summer/
9,Triangle Startup Factory,1010,http://trianglestartupfactory.com/


In [12]:
#function to fetch details from an accelorator

def get_details(id):
    temp= []
    response = requests.get("https://seed-db.com/accelerators/viewall?acceleratorid="+id)  
    txt = response.text
    soup = BeautifulSoup(txt, 'html.parser')
    table=soup.find(id="seedcos")
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    for row in rows:
        columns = row.find_all('td')
        temp.append(columns)
        
    finder = ['button','strong','a','span','span','span', 'span']
    for item in temp:
        for i in range(len(finder)+1):
            if i==(len(finder)):
                item.append(soup.find(itemprop = "name").get_text())
            elif i==1:
                a = item[i].find('a')
                link_text = a['href']
                bid = link_text.split("organization/")
                if(len(bid)==2):
                    item.append(bid[1])
                else:
                    item.append("None")
                item[i] = item[i].find(finder[i]).get_text()
            elif i==5 and item[i].find(finder[i]) is None:
                item[i] = item[i].get_text()    
            elif item[i].find(finder[i]) is None and i!=(len(finder)):
                item[i] = 'None'
            else:
                item[i] = item[i].find(finder[i]).get_text()
            
    return temp
    
get_details("1011")

[['Exited',
  'Rapportive ',
  'http://rapportive.com',
  '6/2010',
  '          15,000,000',
  'H',
  '           1,000,000',
  'rapportive',
  'Y Combinator'],
 ['Exited',
  'Snipshot ',
  'http://snipshot.com',
  '1/2006',
  'None',
  '\n',
  'None',
  'snipshot',
  'Y Combinator'],
 ['None',
  'Clustrix ',
  'http://www.clustrix.com',
  '1/2006',
  'None',
  '\n',
  '          71,650,000',
  'clustrix',
  'Y Combinator'],
 ['Exited',
  'Xobni ',
  'http://www.xobni.com',
  '6/2006',
  '          48,000,000',
  'H',
  '          41,752,000',
  'xobni',
  'Y Combinator'],
 ['Dead',
  'Kirkland North ',
  'http://www.playturf.net',
  '1/2008',
  'None',
  '\n',
  '             225,000',
  'kirkland-north',
  'Y Combinator'],
 ['Dead',
  'Voxli ',
  'http://www.voxli.com',
  '1/2009',
  'None',
  '\n',
  '              15,000',
  'voxli',
  'Y Combinator'],
 ['Exited',
  'reMail ',
  'http://www.remail.com',
  '1/2009',
  '           3,000,000',
  'L',
  'None',
  'remail',
  'Y Combin

In [13]:
#Verify again that one works. 
get_details("1001")

[['None',
  'incrwd ',
  'http://www.myincrwd.com',
  '6/2011',
  'None',
  '\n',
  'None',
  'incrwd',
  '10-xcelerator'],
 ['None',
  'morphCARD ',
  'http://www.morphcard.com',
  '1/2012',
  'None',
  '\n',
  '              20,000',
  'morphcard',
  '10-xcelerator'],
 ['None',
  'Flare Code ',
  'http://www.flarecode.com',
  '6/2011',
  'None',
  '\n',
  '             265,000',
  'flare-code',
  '10-xcelerator'],
 ['Dead',
  'tackk ',
  'http://tackk.com',
  '1/2012',
  'None',
  '\n',
  '           1,800,000',
  'tackk',
  '10-xcelerator'],
 ['None',
  'Mondokio ',
  'http://www.mondokio.com',
  '1/2012',
  'None',
  '\n',
  'None',
  'mondokio',
  '10-xcelerator'],
 ['None',
  'Rewardster',
  'http://rewardster.com',
  '6/2011',
  'None',
  '\n',
  'None',
  'None',
  '10-xcelerator'],
 ['Exited',
  'LaunchGram ',
  'http://www.launchgram.com',
  '1/2012',
  'None',
  '\n',
  '              70,000',
  'launchgram',
  '10-xcelerator'],
 ['None',
  'Zoopshop ',
  'http://zoopshop.co

In [15]:
#fetching data for all accelerators and writing it into a csv file
flag=0
for i in range(0,data.shape[0]):
    aid=str(data.loc[i,1]).strip()
    print("Parcing data for :", aid, "\n"  )
    value=get_details(aid)
    with open('accelorator_data.csv', 'a', newline='') as outfile:
        w = csv.writer(outfile)
        if flag==0:
            w.writerow(['State','Company Name', 'Website & Crunchbase Links', 'Cohort Date', 'Exit Value','Empty  Column', 'Funding','Crunchbase ID', 'Accelerator Name/Program'])
            flag=1
        for val in value:
            w.writerow(val)

Parcing data for : 1001 

Parcing data for : 1002 

Parcing data for : 1003 

Parcing data for : 1004 

Parcing data for : 1005 

Parcing data for : 1006 

Parcing data for : 1007 

Parcing data for : 1008 

Parcing data for : 1009 

Parcing data for : 1010 

Parcing data for : 1011 

Parcing data for : 2001 



ChunkedEncodingError: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')",)', OSError("(10054, 'WSAECONNRESET')",))